In [8]:
# PREPARING OUR KERNEL SESSION TO LOAD SERIALIZED MODELS

import torch
import joblib
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import os
import json
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import joblib
import rasterio
from json import JSONEncoder
from rasterio.crs import CRS


# custom definitions
from src.classes.VGGUdeaSpectral import VGGUdeaSpectral
from src.classes.MultipleRegressionModel import MultipleRegressionModel


def rmse_score(net, X, y):
    y_pred = net.predict(X)
    rmse = (mean_squared_error(y_true=y, y_pred=y_pred)) ** 0.5
    return -rmse  # Skorch tries to maximize the score, so negate the RMSE

In [4]:
from src.classes.VGGUdeaSpectral import VGGUdeaSpectral


vgg_model = joblib.load("src/trained_models/VGGUdeaSpectral1/VGGUdeaSpectral_model1.joblib")

vgg_model

RandomizedSearchCV(cv=5,
                   estimator=<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class 'src.classes.VGGUdeaSpectral.VGGUdeaSpectral'>,
  module__num_bands=3,
),
                   n_iter=50,
                   param_distributions={'lr': [0.01, 0.001, 0.0001],
                                        'max_epochs': [5, 10, 20],
                                        'module__activation_type': ['relu',
                                                                    'sigmoid',
                                                                    'tanh'],
                                        'module__dropout_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x777e749d6fb0>,
                                        'module__fc1_out_features': [512, 1024,
                                                                     2048],
                                        'module__fc2_out_features': [256, 512,
                                                                     1024],
                                        'module__num_filters1': [32, 64, 128],
                                        'module__num_filters2': [64, 128, 256],
                                        'module__num_filters3': [128, 256,
                                                                 512]},
                   scoring='neg_mean_absolute_error', verbose=3)

In [10]:
def read_tif_file(filepath):
    with rasterio.open(filepath) as src:
        return np.array(src.read()), src.meta

def save_prediction_tile(prediction, metadata, output_dir, filename):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a new 100x100 tile filled with the prediction value
    prediction_tile = np.full((100, 100), prediction, dtype=np.float32)
    
    # Update metadata for the tile
    new_meta = metadata.copy()
    new_meta.update({
        'dtype': 'float32',
        'count': 1,
        'driver': 'GTiff'
    })
    
    # Save the tile to a TIFF file
    with rasterio.open(os.path.join(output_dir, filename), 'w', **new_meta) as dest:
        dest.write(prediction_tile, 1)



class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, CRS):
            return obj.to_string()
        # Let the base class default method raise the TypeError
        return JSONEncoder.default(self, obj)

def save_metadata(metadata_list, save_path):
    with open(save_path, 'w') as f:
        json.dump(metadata_list, f, cls=CustomEncoder, indent=4)





In [11]:
# Main directory for sentinel data
sentinel_dir = 'data/sentinel2rgbmedian2020.py'  # Adjust your directory

# Extracting directory name for storing predictions
base_name = os.path.basename(sentinel_dir).replace('.py', '')
prediction_tiles_dir = f'prediction_tiles/prediction_{base_name}'

# Ensure the directory for prediction tiles exists
os.makedirs(prediction_tiles_dir, exist_ok=True)

metadata_list = []
filenames = os.listdir(os.path.dirname(sentinel_dir))

for filename in tqdm(filenames):
    sentinel_path = os.path.join(os.path.dirname(sentinel_dir), filename)
    sentinel_img, sentinel_meta = read_tif_file(sentinel_path)
    
    # Assuming your vgg_model is defined and loaded correctly
    image_tensor = torch.tensor(sentinel_img).to(dtype=torch.float32)
    prediction = vgg_model.predict(image_tensor.unsqueeze(0))  # Predict the coverage
    
    # Save the prediction as a tile
    save_prediction_tile(prediction, sentinel_meta, prediction_tiles_dir, f'pred_{filename}')
    
    # Store metadata
    metadata_list.append({'filename': f'pred_{filename}', 'sentinel_meta': sentinel_meta})

# Save the metadata to a JSON file
save_metadata(metadata_list, os.path.join(prediction_tiles_dir, 'metadata_test.json'))

print("Data preparation, prediction, and serialization complete.")

# Build a VRT from the prediction tiles
os.system(f"gdalbuildvrt {os.path.join(prediction_tiles_dir, 'predictions.vrt')} {os.path.join(prediction_tiles_dir, '*.tif')}")

PermissionError: [Errno 13] Permission denied: '/prediction_tiles'